In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras.applications.vgg19 import VGG19
import numpy as np
import pandas as pd
import PIL.Image as Image
import os
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications.densenet import DenseNet121

In [42]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/BUID Dataset.zip', 'r') as zipObj:

     zipObj.extractall('BreastCancer')

In [43]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import splitfolders
input_folder = '/content/BreastCancer/BUID/'

In [45]:
splitfolders.ratio(input_folder, output="dataset3",
                   seed=123, ratio=(.8, .1, .1), 
                   group_prefix=None)

Copying files: 780 files [00:00, 1320.62 files/s]


In [46]:
BATCH_SIZE = 64
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS = 20

In [47]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/train',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/test',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/val',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

Found 623 files belonging to 3 classes.
Found 80 files belonging to 3 classes.
Found 77 files belonging to 3 classes.


In [48]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/dataset3/train')

class_names= train_dataset.class_names
class_names

Found 623 files belonging to 3 classes.


['benign', 'malignant', 'normal']

In [49]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [50]:
data_augmentation = tf.keras.Sequential([
layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
layers.experimental.preprocessing.RandomRotation(0.2),
])

In [51]:
training_ds = train_ds.map(
lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [52]:
DenseNet1211 = keras.applications.DenseNet121(input_shape=(224,224,3),include_top=False,weights='imagenet')
DenseNet1211.output

<KerasTensor: shape=(None, 7, 7, 1024) dtype=float32 (created by layer 'relu')>

In [53]:
for layer in DenseNet1211.layers:
  DenseNet1211.trainable = False

In [54]:
x= Flatten()(DenseNet1211.output)
prediction=Dense(3,activation='softmax')(x)
model=Model(inputs=DenseNet1211.input,outputs=prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [55]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [56]:
import time
start = time.time()
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)
print("Total time: ", time.time() - start, "seconds")

Epoch 1/20
10/10 [==============================] - 17s 752ms/step - loss: 19.3234 - accuracy: 0.4559 - val_loss: 9.7778 - val_accuracy: 0.5974
Epoch 2/20
10/10 [==============================] - 6s 319ms/step - loss: 5.5444 - accuracy: 0.6645 - val_loss: 5.2110 - val_accuracy: 0.5974
Epoch 3/20
10/10 [==============================] - 7s 269ms/step - loss: 2.8612 - accuracy: 0.7769 - val_loss: 3.3237 - val_accuracy: 0.6883
Epoch 4/20
10/10 [==============================] - 5s 271ms/step - loss: 1.1295 - accuracy: 0.8684 - val_loss: 2.7994 - val_accuracy: 0.7662
Epoch 5/20
10/10 [==============================] - 5s 269ms/step - loss: 0.4880 - accuracy: 0.9133 - val_loss: 2.1590 - val_accuracy: 0.7792
Epoch 6/20
10/10 [==============================] - 5s 271ms/step - loss: 0.3086 - accuracy: 0.9358 - val_loss: 2.0729 - val_accuracy: 0.7532
Epoch 7/20
10/10 [==============================] - 5s 269ms/step - loss: 0.1551 - accuracy: 0.9599 - val_loss: 1.8688 - val_accuracy: 0.7532
Epoc

In [57]:
scores = model.evaluate(test_ds)

2/2 [==============================] - 1s 61ms/step - loss: 2.1706 - accuracy: 0.7750
